## Problem description
Given a set of cities $\{c_1, c_2, ..., c_N\}$, find the path $P\in\mathbb{N}^N$ (where each $P_i$ is the index of a city) that passes through all the cities with the least total travelling distance.


## Constrained quadratic formulation
The Travellig Salesman problem can be modeled as the following constrained quadratic optimization model:

$$x^* = \min_{x \in \mathbb{B}^{N\times N}} \sum_{p=2}^N \sum_{i=1}^N \sum_{j \neq i }^N d_{(i,j)} x_{(i,p)} x_{(j,p-1)}, \qquad \forall_{1 \leq i \leq N}: \sum_{p=1}^N x^*_{(i,p)} = 1$$

where:
* $x_{(i,p)} \in \mathbb{B}$ - indicates that the i-th city of the map is allocated in the p-th position of the path;
* $d_{(i,j)} \in \mathbb{R}$ - distance between i-th and j-th cities.

Once $x^*$ is found, the path $P=[P_1, P_2, ..., P_N]$ is defined as $P_p = \max_{0 \leq i \leq N} x_{(i,p)}$.


## Converting to QUBO

To convert the 1th equation to a QUBO model, we can transform the constrait into a quadratic penalty term $H(x)$:

$$H(x) = \sum_{i=1}^N \left( \sum_{p=1}^N x_{(i,p)} - 1 \right)^2 = \sum_{i=1}^N \left( \sum_{p=1}^N \sum_{q=1}^N x_{(i,p)} x_{(i,q)} - 2 \sum_{p=1}^N x_{(i,p)} + 1\right) = \sum_{i=1}^N \sum_{p=1}^N \sum_{q=1}^N x_{(i,p)}  x_{(i,q)} - 2 \sum_{i=1}^N \sum_{p=1}^N x_{(i,p)} + N$$

Since the constant $N$ does not change the final result of the optimization, we can simply cut it out of the term.

$$H(x) = \sum_{i=1}^N \sum_{p=1}^N \sum_{q=1}^N x_{(i,p)}  x_{(i,q)} - 2 \sum_{i=1}^N \sum_{p=1}^N x_{(i,p)}$$

The final objective function is:

$$L(x) = \sum_{p=2}^N \sum_{i=1}^N \sum_{j \neq i}^N d_{(i,j)} x_{(i,p)} x_{(j,p-1)} + \gamma H(x)$$

where $\gamma \in \mathbb{R}$ is a penalty factor.

In [ ]:
import numpy as np

In [ ]:
N = 5 # Number of cities

In [ ]:
# -- Generate distances ---
d = np.zeros(shape=(N,N))
for i in range(N):
    for j in range(i+1, N):
    d_i_j = np.random.rand()
    d[i,j] = d_i_j
    d[j,i] = d_i_j

# I know there are more efficient ways to create a symmetric matrix with numpy. This code is for ease of understanding.

In [ ]:
# --- Generate variables ---
x = np.zeros(shape=(N,N))
count = 0
for i in range(N):
    for p in range(N):
        x[i,p] = count
        count += 1

In [ ]:
# --- Generate the QUBO matrix ---
qubo = {(a,b):0 for a in range(N) for b in range(N)}
gamma = 10

st = lambda a, b: (a, b) if a <= b else (b, a) # sorted tuple

# main term
for p in range(1, N):
    for i in range(N):
        for j in range(N):
            if i != j:
                qubo[st(x[i,p], x[j,p-1])] += d[i,j]

# penalty terms
for i in range(N):
    for p in range(N):
        for q in range(N):
            qubo[st(x[i,p], x[i,q])] += gamma


for i in range(N):
    for p in range(N):
        qubo[st(x[i,p], x[i,p])] -= 2*gamma